In [2]:
import pandas as pd
import json
import numpy as np
from collections import defaultdict

In [3]:
#Read .csv dataset file
df_data = pd.read_csv('hospital-inpatient-discharges-pneumonia.csv', low_memory=False)

In [4]:
#columns_to_drop = ["Facility Name","Ethnicity","Health Service Area","Operating Certificate Number","Facility ID","Zip Code - 3 digits",
#                 "CCS Diagnosis Code","CCS Procedure Code","APR DRG Code","APR MDC Code","APR Severity of Illness Code",
#                 "Source of Payment 1","Source of Payment 2","Source of Payment 3","Attending Provider License Number",
#                 "Operating Provider License Number","Other Provider License Number","Payment Typology 1",
#                 "Payment Topology 2","Payment Typology 2","Payment Typology 3","Facility Id","Ratio of Total Costs to Total Charges",
#                  "Discharge Year","Total Charges","Total Costs","Abortion Edit Indicator","Birth Weight"]
columns_to_drop = ["Facility Name","Ethnicity","Health Service Area","Operating Certificate Number","Facility ID","Zip Code - 3 digits",
                 "CCS Diagnosis Code","CCS Procedure Code","APR DRG Code","APR MDC Code","APR Severity of Illness Code",
                 "Source of Payment 1","Source of Payment 2","Source of Payment 3","Attending Provider License Number",
                 "Operating Provider License Number","Other Provider License Number","Payment Typology 1",
                 "Payment Topology 2","Payment Typology 2","Payment Typology 3","Facility Id","Ratio of Total Costs to Total Charges",
                  "Discharge Year","Total Charges","Total Costs","Abortion Edit Indicator","Birth Weight"]
df_data.drop(columns=columns_to_drop, inplace=True)

In [18]:
#df_data.loc[df_data["APR Severity of Illness Description"].isnull() & (df_data["APR Severity of Illness Code"] == 1), "APR Severity of Illness Description"] = "Minor"
#df_data.loc[df_data["APR Severity of Illness Description"].isnull() & (df_data["APR Severity of Illness Code"] == 2), "APR Severity of Illness Description"] = "Moderate"
#df_data.loc[df_data["APR Severity of Illness Description"].isnull() & (df_data["APR Severity of Illness Code"] == 3), "APR Severity of Illness Description"] = "Major"
#df_data.loc[df_data["APR Severity of Illness Description"].isnull() & (df_data["APR Severity of Illness Code"] == 4), "APR Severity of Illness Description"] = "Extreme"

In [5]:
nan_counts = df_data.isnull().sum()

# Total number of NaN values in the entire DataFrame

print("Number of NaN values in each column:")
print(nan_counts)

Number of NaN values in each column:
Hospital County                        9
Age Group                              0
Gender                                 0
Race                                   0
Length of Stay                         0
Type of Admission                      0
Patient Disposition                    6
CCS Diagnosis Description              0
CCS Procedure Description              0
APR DRG Description                    0
APR MDC Description                    0
APR Severity of Illness Description    2
APR Risk of Mortality                  2
APR Medical Surgical Description       0
Emergency Department Indicator         0
dtype: int64


In [6]:
df_data.dropna(inplace=True)

In [26]:
# Function to build the file in correct JSONL format
def create_message(row):
    message_system = {
        "role": "system",
        "content": '"Given characteristics about a patient, where applicable, provide the APR Severity of Illness Description: "'
                   '"Hospital County", "Age Group", "Gender", "Race", "Length of Stay", "Type of Admission", "Patient Disposition", '
                   '"CCS Diagnosis Description", "CCS Procedure Description", "APR DRG Description", "APR MDC Description",'
                   '"APR Severity of Illness Code", "APR Risk of Mortality", "APR Medical Surgical Description",'
                    '"Emergency Department Indicator".'
    }
    
    message_user = {
        "role": "user",
        "content": f"Sources: {row['Hospital County']}, {row['Age Group']}, {row['Gender']}, {row['Race']}, {row['Length of Stay']},"
                   f" {row['Type of Admission']},{row['Patient Disposition']}, {row['CCS Diagnosis Description']},"
                   f"{row['CCS Procedure Description']},{row['APR DRG Description']}, {row['APR MDC Description']},"
                   f"{row['APR Risk of Mortality']}, {row['APR Medical Surgical Description']},"
                   f"{row['Emergency Department Indicator']}"
    }
    
    message_assistant = {
        "role": "assistant",
        "content": f"{output_column}: {row[output_column]}"
    }
    
    return {"messages": [message_system, message_user, message_assistant]}

# List to store all messages
messages = []

# Iterate over the rows of the DataFrame, starting from the second row (index 1) and limiting to the 101st row (index 100)
for index, row in df_data.iloc[1:101].iterrows():
    
    # Create a message for each row
    message = create_message(row)
    # Add the messages to the list
    messages.append(message)

# Save the messages in a JSONL file
with open('output.jsonl', 'w') as f:
    for msg in messages:
        f.write(json.dumps(msg) + '\n') 


In [17]:
def create_message(context_phrase, row, df_columns, output_column):
    # Initialize an empty dictionary to store the message
    message = {"messages": []}
    
    # Create the system message
    system_message_content = {
        "role": "system",
        "content": f"{context_phrase}" + ', '.join(df_columns)
    }
    # Add the system message to the list of messages in the dictionary
    message["messages"].append(system_message_content)
    
    user_message = {"role": "user", "content": ""}

    # Iterate over each column and value of the Dataframe line
    for column, value in row.items():
        # Adiciona o nome da coluna e o valor correspondente à mensagem
        user_message["content"] += f"{column}: {value} , "

    # Remove the last character "," of the message
    user_message["content"] = user_message["content"][:-2]
    
    message["messages"].append(user_message)

    message_assistant = {
        "role": "assistant",
        "content": f"{output_column}: {row[output_column]}"
    }
    message["messages"].append(message_assistant)
    
    return message

# List to store all messages
messages = []

phrase_context = "Given characteristics about a patient, where applicable, provide the APR Severity of Illness Description: "
# Columns for the system message
system_message_columns = df_data.columns.tolist()
output_column = "APR Severity of Illness Description"

# Iterate over the rows of the DataFrame
for index, row in df_data.iloc[1:101].iterrows():
    # Create a message for each row
    message = create_message(phrase_context, row, system_message_columns, output_column)
    # Add the message to the list
    messages.append(message)

# Save the messages in a JSONL file
with open('output.jsonl', 'w') as f:
    for msg in messages:
        f.write(json.dumps(msg) + '\n')

In [18]:
data_path = "output.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 100
First example:
{'role': 'system', 'content': 'Given characteristics about a patient, where applicable, provide the APR Severity of Illness Description: Hospital County, Age Group, Gender, Race, Length of Stay, Type of Admission, Patient Disposition, CCS Diagnosis Description, CCS Procedure Description, APR DRG Description, APR MDC Description, APR Severity of Illness Description, APR Risk of Mortality, APR Medical Surgical Description, Emergency Department Indicator'}
{'role': 'user', 'content': 'Hospital County: Albany , Age Group: 70 or Older , Gender: F , Race: White , Length of Stay: 3 , Type of Admission: Emergency , Patient Disposition: Inpatient Rehabilitation Facility , CCS Diagnosis Description: PNEUMONIA , CCS Procedure Description: NO PROC , APR DRG Description: OTHER PNEUMONIA , APR MDC Description: Diseases and Disorders of the Respiratory System , APR Severity of Illness Description: Major , APR Risk of Mortality: Moderate , APR Medical Surgical Descript

In [19]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found
